### Exploration

In [1]:
!git clone https://github.com/openai/baselines.git
%cd baselines
!pip install -e .
!pip install --upgrade tensorflow==2.2.0
!pip install gym

fatal: destination path 'baselines' already exists and is not an empty directory.
/project/baselines
Obtaining file:///project/baselines
  Attempting uninstall: baselines
    Found existing installation: baselines 0.1.6
    Uninstalling baselines-0.1.6:
      Successfully uninstalled baselines-0.1.6
  Running setup.py develop for baselines
     |████████████████████████████████| 516.2 MB 6.1 kB/s  eta 0:00:01    |▉                               | 13.7 MB 17.4 MB/s eta 0:00:29     |█▍                              | 22.4 MB 17.4 MB/s eta 0:00:29     |██                              | 30.8 MB 17.4 MB/s eta 0:00:28     |██▎                             | 37.3 MB 17.4 MB/s eta 0:00:28     |███▏                            | 50.4 MB 15.7 MB/s eta 0:00:30     |███▎                            | 52.5 MB 15.7 MB/s eta 0:00:30     |████                            | 63.3 MB 15.7 MB/s eta 0:00:29     |████                            | 65.2 MB 15.7 MB/s eta 0:00:29     |████▏                          

In [3]:
import time
import gym
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

# from baselines.ppo2 import ppo2
# from baselines.common.vec_env.dummy_vec_env import DummyVecEnv

# from baselines import deepq
# from baselines import bench
# from baselines import logger
import tensorflow as tf

# from baselines.common.tf_util import make_session

In [4]:
data = pd.read_csv("/project/datasets/clean-ids-collection/cic-ddos2019/clean/cicddos2019.csv")
data.head(5)

,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,2018-11-03 09:18:16.964447,114456999,45,0,0.0,0.0,0.0,0.0,0.000000,...,0,8185.5835,28337.111,98168.0,3.0,9529897.0,351582.62,10001143.0,9048097.0,Benign
1,0,2018-11-03 09:18:18.506537,114347504,56,0,0.0,0.0,0.0,0.0,0.000000,...,0,35028.4180,121314.914,420255.0,4.0,9493930.0,351541.10,9978130.0,8820294.0,Benign
2,6,2018-11-03 09:18:18.610576,36435473,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62416.0000,0.000,62416.0,62416.0,36373056.0,0.00,36373056.0,36373056.0,Benign
3,6,2018-11-03 09:18:18.610579,36434705,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62413.0000,0.000,62413.0,62413.0,36372292.0,0.00,36372292.0,36372292.0,Benign
4,6,2018-11-03 09:18:18.610581,36434626,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62409.0000,0.000,62409.0,62409.0,36372216.0,0.00,36372216.0,36372216.0,Benign


In [5]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138007 entries, 0 to 138006
Data columns (total 69 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Protocol                  138007 non-null  int64  
 1   Timestamp                 138007 non-null  object 
 2   Flow Duration             138007 non-null  int64  
 3   Total Fwd Packets         138007 non-null  int64  
 4   Total Backward Packets    138007 non-null  int64  
 5   Fwd Packets Length Total  138007 non-null  float64
 6   Bwd Packets Length Total  138007 non-null  float64
 7   Fwd Packet Length Max     138007 non-null  float64
 8   Fwd Packet Length Min     138007 non-null  float64
 9   Fwd Packet Length Mean    138007 non-null  float64
 10  Fwd Packet Length Std     138007 non-null  float64
 11  Bwd Packet Length Max     138007 non-null  float64
 12  Bwd Packet Length Min     138007 non-null  float64
 13  Bwd Packet Length Mean    138007 non-null  f

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Act Data Packets,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,138007.000000,1.380070e+05,138007.000000,138007.000000,1.380070e+05,1.380070e+05,138007.000000,138007.000000,138007.000000,138007.000000,...,138007.000000,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05
mean,11.592050,1.035663e+07,13.812169,6.636576,4.019830e+03,5.102600e+03,249.273218,132.853515,161.376744,37.345936,...,10.525227,-1.175840e+07,7.345821e+04,3.066152e+04,1.140858e+05,5.479838e+04,3.445620e+06,3.282404e+05,3.770891e+06,3.153606e+06
std,5.607782,2.853154e+07,70.298079,99.503208,5.151998e+04,1.880790e+05,419.156188,234.493611,241.043532,116.216344,...,62.973954,1.112129e+08,5.176193e+05,2.392317e+05,6.900226e+05,4.802156e+05,1.157949e+07,2.158705e+06,1.230392e+07,1.120812e+07
min,0.000000,1.000000e+00,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,-1.408238e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000,1.178000e+03,2.000000,0.000000,1.800000e+01,0.000000e+00,6.000000,0.000000,6.000000,0.000000,...,1.000000,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,17.000000,2.489200e+04,3.000000,2.000000,8.600000e+01,0.000000e+00,43.000000,30.000000,39.000000,0.000000,...,1.000000,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,17.000000,2.999380e+06,8.000000,2.000000,2.064000e+03,1.660000e+02,440.000000,152.000000,349.500000,14.433757,...,5.000000,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,17.000000,1.199987e+08,20444.000000,31700.000000,1.526642e+07,5.842950e+07,32120.000000,2131.000000,3015.290500,2221.556200,...,18766.000000,1.480000e+03,4.050800e+07,1.100562e+07,4.050800e+07,4.050800e+07,1.192194e+08,6.961402e+07,1.192194e+08,1.192194e+08


Preprocessing done by Laurens D'Hooge

## Creating the model
- Build an agent to classify network flow automatically
- Feed a packet that gets classified
- Want the classification to be equal to the label

In [6]:
def balancing_dataset_sampler_df(df, train_frac=0.2, val_frac=0.1, test_frac=0.7):
    col = df.columns[-1]
    cols = df.columns[:-1]
    vc = df[col].value_counts()
    n = vc.iloc[-1]
    m = vc.iloc[0]
    initial_cut = df.loc[df[col] == vc.index[0]].sample(n=int(m-n), replace=False)
    df = df.drop(index=initial_cut.index)

    train_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*train_frac), replace=False))
    train_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=train_df.index)

    validation_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*val_frac), replace=False))
    validation_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=validation_df.index)

    test_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*test_frac), replace=False))
    test_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=test_df.index)

    return train_df[cols], train_df[col], validation_df[cols], validation_df[col], test_df[cols], test_df[col]


In [7]:
data.drop('Timestamp', inplace=True, axis=1)
print(data['Label'].value_counts())
data['Label'] = data['Label'].astype('object')
atk_idx = data.loc[data['Label'] != "Benign"].index
data.loc[atk_idx, 'Label'] = 1.0
data.loc[data.index.difference(atk_idx), 'Label'] = 0.0
data['Label'] = data['Label'].astype(dtype=np.float32)

x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(data)

Benign           95095
DrDoS_NTP        13687
TFTP             13027
Syn               6713
UDP               2158
MSSQL             1763
DrDoS_UDP         1174
UDP-lag           1097
DrDoS_MSSQL        927
DrDoS_DNS          552
LDAP               484
DrDoS_SSDP         413
DrDoS_SNMP         392
Portmap            215
DrDoS_LDAP         174
NetBIOS             73
DrDoS_NetBIOS       51
UDPLag              10
WebDDoS              2
Name: Label, dtype: int64


In [8]:
print("Training")
print(x_train.shape)
print(y_train.shape)
print("Validation")
print(x_val.shape)
print(y_val.shape)
print("Testing")
print(x_test.shape)
print(y_test.shape)


Training
(17164, 67)
(17164,)
Validation
(8582, 67)
(8582,)
Testing
(60076, 67)
(60076,)


In [9]:
# all attacks
# attacks_dict = {
#     'Benign': 0,
#     'Portmap': 1,
#     'NetBIOS': 2,
#     'LDAP': 3,
#     'MSSQL': 4,
#     'UDP': 5,
#     'UDPLag': 6,
#     'Syn': 7,
#     'DrDoS_NTP': 8,
#     'DrDoS_DNS': 9,
#     'DrDoS_LDAP': 10,
#     'DrDoS_MSSQL': 11,
#     'DrDoS_NetBIOS': 12,
#     'DrDoS_SNMP': 13,
#     'DrDoS_SSDP': 14,
#     'DrDoS_UDP': 15,
#     'UDP-lag': 16,
#     'WebDDoS': 17,
#     'TFTP': 18,
# }


In [10]:
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
x_val = np.expand_dims(x_val, -1)

In [11]:
y_train = np.expand_dims(y_train, -1)
y_test = np.expand_dims(y_test, -1)

### Custom environment

In [23]:
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        # change to 19 if detectiong all different attacks
        self.action_space = gym.spaces.Discrete(2)
        # All the features we have, 67 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(67,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)

        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y[next_obs_idx])
            obs = self.x[next_obs_idx]

        else:
            obs = self.x[self.dataset_idx]
            self.expected_action = int(self.y[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()

        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

In [24]:
env = IdsEnv(images_per_episode=4096)

In [25]:
class DDDQN(tf.keras.Model):
    def __init__(self):
      super(DDDQN, self).__init__()
      self.d1 = tf.keras.layers.Dense(128, activation='relu')
      self.d2 = tf.keras.layers.Dense(128, activation='relu')
      self.v = tf.keras.layers.Dense(1, activation=None)
      self.a = tf.keras.layers.Dense(env.action_space.n, activation=None)

    def call(self, input_data):
      x = self.d1(input_data)
      x = self.d2(x)
      v = self.v(x)
      a = self.a(x)
      Q = v +(a -tf.math.reduce_mean(a, axis=1, keepdims=True))
      return Q

    def advantage(self, state):
      x = self.d1(state)
      x = self.d2(x)
      a = self.a(x)
      return a

In [32]:
class exp_replay():
    def __init__(self, buffer_size= 1000000):
        self.buffer_size = buffer_size
        self.state_mem = np.zeros((self.buffer_size, *(env.observation_space.shape)), dtype=np.float32)
        self.action_mem = np.zeros((self.buffer_size), dtype=np.int32)
        self.reward_mem = np.zeros((self.buffer_size), dtype=np.float32)
        self.next_state_mem = np.zeros((self.buffer_size, *(env.observation_space.shape)), dtype=np.float32)
        self.done_mem = np.zeros((self.buffer_size), dtype=np.bool)
        self.pointer = 0

    def add_exp(self, state, action, reward, next_state, done):
        idx  = self.pointer % self.buffer_size 
        self.state_mem[idx] = state.flatten()
        self.action_mem[idx] = action.flatten()
        self.reward_mem[idx] = reward
        self.next_state_mem[idx] = next_state.flatten()
        self.done_mem[idx] = 1 - int(done)
        self.pointer += 1

    def sample_exp(self, batch_size= 64):
        max_mem = min(self.pointer, self.buffer_size)
        batch = np.random.choice(max_mem, batch_size, replace=False)
        states = self.state_mem[batch]
        actions = self.action_mem[batch]
        rewards = self.reward_mem[batch]
        next_states = self.next_state_mem[batch]
        dones = self.done_mem[batch]
        return states, actions, rewards, next_states, dones

In [33]:
class agent():
      def __init__(self, gamma=0.99, replace=100, lr=0.001):
          self.gamma = gamma
          self.epsilon = 1.0
          self.min_epsilon = 0.01
          self.epsilon_decay = 1e-3
          self.replace = replace
          self.trainstep = 0
          self.memory = exp_replay()
          self.batch_size = 64
          self.q_net = DDDQN()
          self.target_net = DDDQN()
          opt = tf.keras.optimizers.Adam(learning_rate=lr)
          self.q_net.compile(loss='mse', optimizer=opt)
          self.target_net.compile(loss='mse', optimizer=opt)


      def act(self, state):
          if np.random.rand() <= self.epsilon:
              return np.random.choice([i for i in range(env.action_space.n)])

          else:
              actions = self.q_net.advantage(np.array([state]))
              action = np.argmax(actions)
              return action


      
      def update_mem(self, state, action, reward, next_state, done):
          self.memory.add_exp(state, action, reward, next_state, done)


      def update_target(self):
          self.target_net.set_weights(self.q_net.get_weights())     

      def update_epsilon(self):
          self.epsilon = self.epsilon - self.epsilon_decay if self.epsilon > self.min_epsilon else self.min_epsilon
          return self.epsilon

          
      def train(self):
          if self.memory.pointer < self.batch_size:
             return 
          
          if self.trainstep % self.replace == 0:
             self.update_target()
          states, actions, rewards, next_states, dones = self.memory.sample_exp(self.batch_size)
          target = self.q_net.predict(states)
          next_state_val = self.target_net.predict(next_states)
          max_action = np.argmax(self.q_net.predict(next_states), axis=1)
          batch_index = np.arange(self.batch_size, dtype=np.int32)
          q_target = np.copy(target)
          q_target[batch_index, actions] = rewards + self.gamma * next_state_val[batch_index, max_action]*dones
          self.q_net.train_on_batch(states, q_target)
          self.update_epsilon()
          self.trainstep += 1

      def save_model(self):
          self.q_net.save("model.h5")
          self.target_net.save("target_model.h5")


      def load_model(self):
          self.q_net = load_model("model.h5")
          self.target_net = load_model("model.h5")

In [34]:
agentoo7 = agent()
steps = 400

for s in range(steps):
  done = False
  state = env.reset()
  total_reward = 0
  while not done:
    #env.render()
    action = agentoo7.act(state)
    next_state, reward, done, _ = env.step(action)
    agentoo7.update_mem(state, action, reward, next_state, done)
    agentoo7.train()
    state = next_state
    total_reward += reward
    
    if done:
      print("total reward after {} episode is {} and epsilon is {}".format(s, total_reward, agentoo7.epsilon))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



InvalidArgumentError: Matrix size-incompatible: In[0]: [67,1], In[1]: [67,128] [Op:MatMul]

Train classifier using dqn

In [ ]:
def ids_dqn_eval(dqn_model):
    attempts, correct = 0,0

    env = IdsEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, _ = env.step(dqn_model(obs[None])[0])

                attempts += 1
                if rew > 0:
                    correct += 1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))

ids_dqn_eval(dqn_model)